chatbot

In [1]:
from typing_extensions import TypedDict

from langgraph.graph import START, END, StateGraph

# reduce
from typing import Annotated
from langgraph.graph.message import add_messages

Load API key bằng `dotenv`

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # This will load all environment variables from .env file

# Make sure OPENAI_API_KEY is set
os.environ["OPENAI_API_KEY"] = str(os.getenv("OPENAI_API_KEY"))
os.environ["GROQ_API_KEY"] = str(os.getenv("GROQ_API_KEY"))
os.environ["GOOGLE_API_KEY"] = str(os.getenv("GOOGLE_API_KEY"))


Call model

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

llm.invoke("test")

AIMessage(content="Okay, I'm ready.  How can I help you?  What kind of test do you need?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--33373316-eadb-4653-ab8b-715105b3a620-0', usage_metadata={'input_tokens': 1, 'output_tokens': 24, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}})

In [4]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

Tạo Node

In [5]:
def bot(state: State) -> State:
    return State({
		"messages": [llm.invoke(state["messages"])]
	})

In [6]:
# nodes

builder = StateGraph(State)

builder.add_node("bot", bot)

# edges

builder.add_edge(START, "bot")
builder.add_edge("bot", END)



In [10]:
# from IPython import 
from IPython.display import Image, display
graph = builder.compile()
print(graph.get_graph().draw_ascii())
# display(Image(graph.get_graph().draw_ascii()))

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
   +-----+     
   | bot |     
   +-----+     
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [14]:
import json

query = "how to invert a binary tree"
res = graph.invoke(State({
	"messages": [query]
}))

print(res)

{'messages': [HumanMessage(content='how to invert a binary tree', additional_kwargs={}, response_metadata={}, id='ed8eee9d-ddc6-4165-853b-f7a55d9fa30b'), AIMessage(content='Inverting a binary tree means swapping the left and right children of every node in the tree.  There are several ways to do this, but the most common and efficient approaches use recursion or iteration.\n\n**1. Recursive Approach:**\n\nThis is generally considered the most elegant and easiest-to-understand solution.  The core idea is:\n\n* **Base Case:** If the node is `null`, do nothing (we\'ve reached the end of a branch).\n* **Recursive Step:** Swap the left and right children of the current node, then recursively invert the left subtree and the right subtree.\n\nHere\'s Python code implementing this:\n\n```python\nclass Node:\n    def __init__(self, data):\n        self.data = data\n        self.left = None\n        self.right = None\n\ndef invert_tree(root):\n    if root is None:\n        return None\n\n    # S

Stream

In [17]:
import json

query = "how to reverse a linked list"
for event in graph.stream(State({
	"messages": [query]
}), stream_mode="values"):
    print(event)



{'messages': [HumanMessage(content='how to reverse a linked list', additional_kwargs={}, response_metadata={}, id='81b85f7a-d22b-4f42-8875-5ab485d9df02')]}
{'messages': [HumanMessage(content='how to reverse a linked list', additional_kwargs={}, response_metadata={}, id='81b85f7a-d22b-4f42-8875-5ab485d9df02'), AIMessage(content='There are several ways to reverse a linked list. Here are three common approaches, with explanations and code examples in Python:\n\n**Method 1: Iterative Reversal**\n\nThis is the most efficient and commonly used method.  It iterates through the list, changing the `next` pointers one by one.\n\n```python\nclass Node:\n    def __init__(self, data):\n        self.data = data\n        self.next = None\n\ndef reverse_linked_list_iterative(head):\n    prev = None\n    curr = head\n    while curr:\n        next_node = curr.next  # Store the next node\n        curr.next = prev       # Reverse the current node\'s pointer\n        prev = curr           # Move \'prev\' o